In [0]:
DECLARE OR REPLACE VARIABLE catalog_use STRING DEFAULT 'fhir_workshop';
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT REPLACE(SPLIT(current_user(), '@')[0], '.', '_');

In [0]:
SET VARIABLE catalog_use = :catalog_use; 
SET VARIABLE schema_use = :schema_use;